## Control Variables
- Resolution = 0.25
- ROBOT_RADIUS = 0.25
- NUM_OF_AGENT = 5
- BO_INITIAL_SAMPLES = 30
- BO_OPT_SAMPLES = 20

In [1]:
import BayesianOptimisation.boVoronoiDirected as boVoronoiDirected
import BayesianOptimisation.expGrid as expGrid
import BayesianOptimisation.expVoronoi as expVoronoi
import BayesianOptimisation.genVoronoiDirected as genVoronoiDirected
import BayesianOptimisation.experiment_setup as experiment
import BayesianOptimisation.sets_of_exp as sets_of_exp
from core.DataStructure import *

from importlib import reload
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.patches import Rectangle
import copy
import re

In [2]:
from scipy.optimize import minimize, Bounds
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, Reshape
from tensorflow.keras.utils import plot_model
from sklearn.model_selection import train_test_split

In [3]:
np.random.seed(9)

### Collecting viable start-end locations

In [4]:
def get_locations(scene = "./input/random-32-32-10/scen-even/random-32-32-10-even-2.scen"):
    f = open(scene, "r")
    lines = f.readlines()
    f.close()
    
    starts = []
    ends = []
    pairs = {}
    c = 0
    for index, l in enumerate(lines):
        x = l.split()
        match = re.match(r"random-32-32-10", x[1])
        if match:
            coord = np.array(x[4:8]).astype('float') + 1
            starts.append([coord[0], coord[1]])
            ends.append([coord[2], coord[3]])
    return starts, ends

In [5]:
all_starts = []
all_ends = []
for i in range (25):
    scene = "./input/random-32-32-10/scen-even/random-32-32-10-even-" + str(0) + ".scen"
    starts, ends = get_locations(scene)
    all_starts.extend(starts)
    all_ends.extend(ends)

In [6]:
# pd.DataFrame(all_starts).to_csv("./data/gen-starts-ends/starts.csv", index=None)
# pd.DataFrame(all_ends).to_?csv("./data/gen-starts-ends/ends.csv", index=None)

In [7]:
start = pd.read_csv("./data/gen-starts-ends/starts.csv")
start = start.to_numpy()
start = start.reshape((-1,2))

end = pd.read_csv("./data/gen-starts-ends/ends.csv")
end = end.to_numpy()
end = end.reshape((-1,2))

### Generating Data

In [8]:
num_of_agent = 10
x_path = "./data/gen-starts-ends/X.csv"
y_path = "./data/gen-starts-ends/Y.csv"

cur_idx = 0
exp = experiment.Experiment()
exp.setParameters(
    specified_start = start[cur_idx:cur_idx+num_of_agent+19], 
    specified_end = end[cur_idx:cur_idx+num_of_agent+19])
exp_set = sets_of_exp.Experiment_Sets(edges = exp.edges_dir, nodes = exp.nodes)

removed num of points 79
num of points remained 444
Set Hyper Parameters, solving for 10 agents


In [9]:
# Prepare the trainable edges
for loop in range(1):
    print("Generating Sample", loop)
    cur_idx = loop * num_of_agent
    exp = experiment.Experiment()
    exp.setParameters(
        specified_start = start[cur_idx:cur_idx+num_of_agent+19], 
        specified_end = end[cur_idx:cur_idx+num_of_agent+19])
    
    if loop == 0:
        prob, opt, graph, exp_set = genVoronoiDirected.gen_voronoi_directed(exp, exp_set, first_sample = True)
    else:
        prob, opt, graph, exp_set = genVoronoiDirected.gen_voronoi_directed(exp, exp_set, first_sample = False)
    
    X = np.hstack((exp.start_locations,exp.end_locations))
    Y = np.array(prob)
    print("Shape", X.shape, Y.shape)

Generating Sample 0
removed num of points 79
num of points remained 444
Set Hyper Parameters, solving for 10 agents
Number of trainable probabilities 447
Shape (10, 4) (450,)


In [10]:
for loop in range(0, 223):
    print("\nGenerating Sample", loop)
    cur_idx = loop * num_of_agent
    exp = experiment.Experiment()
    exp.setParameters(
        specified_start = start[cur_idx:cur_idx+num_of_agent+19], 
        specified_end = end[cur_idx:cur_idx+num_of_agent+19])
    
    if loop == 0:
        prob, opt, graph, exp_set = genVoronoiDirected.gen_voronoi_directed(exp, exp_set, first_sample = True)
    else:
        prob, opt, graph, exp_set = genVoronoiDirected.gen_voronoi_directed(exp, exp_set, first_sample = False)
    
    X = np.hstack((exp.start_locations,exp.end_locations))
    Y = np.array(prob)
    print("Shape", X.shape, Y.shape)
    
    if X.shape != (10, 4) or Y.shape != (450,):
        print("Wrong shape discard sample")
        continue
        
    if loop == 0:
        with open(x_path, 'a+') as f:
            pd.DataFrame(X).to_csv(f, index=None)
        with open(y_path, 'a+') as f:
            pd.DataFrame(Y).to_csv(f, index=None)
    else:
        with open(x_path, 'a+') as f:
            pd.DataFrame(X).to_csv(f, index=None,header=None)
        with open(y_path, 'a+') as f:
            pd.DataFrame(Y).to_csv(f, index=None,header=None)
    


Generating Sample 0
removed num of points 79
num of points remained 444
Set Hyper Parameters, solving for 10 agents
Number of trainable probabilities 447
Shape (10, 4) (450,)

Generating Sample 1
removed num of points 79
num of points remained 444
Set Hyper Parameters, solving for 10 agents
Number of trainable probabilities 444
last three [-0.45249305  0.42937408  0.04834122]
Shape (10, 4) (450,)

Generating Sample 2
removed num of points 79
num of points remained 444
Set Hyper Parameters, solving for 10 agents
Number of trainable probabilities 435
last three [0.2049061  0.5051465  0.00154229]
Shape (10, 4) (450,)

Generating Sample 3
removed num of points 79
num of points remained 444
Set Hyper Parameters, solving for 10 agents
Number of trainable probabilities 429
last three [0.12430944 0.59744026 0.00883319]
Shape (10, 4) (450,)

Generating Sample 4
removed num of points 79
num of points remained 444
Set Hyper Parameters, solving for 10 agents
Number of trainable probabilities 428


Set Hyper Parameters, solving for 10 agents
Number of trainable probabilities 447
last three [-0.2934247   0.62839133  0.02660475]
Shape (10, 4) (450,)

Generating Sample 37
removed num of points 79
num of points remained 444
Set Hyper Parameters, solving for 10 agents
Number of trainable probabilities 444
last three [-0.1879935   0.73857515  0.02787253]
Shape (10, 4) (450,)

Generating Sample 38
removed num of points 79
num of points remained 444
Set Hyper Parameters, solving for 10 agents
Number of trainable probabilities 435
last three [-0.43871367  0.68210655  0.03822185]
Shape (10, 4) (450,)

Generating Sample 39
removed num of points 79
num of points remained 444
Set Hyper Parameters, solving for 10 agents
Number of trainable probabilities 429
last three [0.10380838 0.74147333 0.04393552]
Shape (10, 4) (450,)

Generating Sample 40
removed num of points 79
num of points remained 444
Set Hyper Parameters, solving for 10 agents
Number of trainable probabilities 428
last three [0.449

Number of trainable probabilities 447
last three [-0.10712814  0.74573868  0.00581032]
Shape (10, 4) (450,)

Generating Sample 73
removed num of points 79
num of points remained 444
Set Hyper Parameters, solving for 10 agents
Number of trainable probabilities 444
last three [-0.15188692  0.5444897   0.04230426]
Shape (10, 4) (450,)

Generating Sample 74
removed num of points 79
num of points remained 444
Set Hyper Parameters, solving for 10 agents
Number of trainable probabilities 435
last three [-0.06368218  0.44575299  0.03839394]
Shape (10, 4) (450,)

Generating Sample 75
removed num of points 79
num of points remained 444
Set Hyper Parameters, solving for 10 agents
Number of trainable probabilities 429
last three [0.18842693 0.68750942 0.02568798]
Shape (10, 4) (450,)

Generating Sample 76
removed num of points 79
num of points remained 444
Set Hyper Parameters, solving for 10 agents
Number of trainable probabilities 428
last three [-0.48177695  0.53180133  0.0315132 ]
Shape (10, 4

Number of trainable probabilities 447
last three [0.30631195 0.26163309 0.0359356 ]
Shape (10, 4) (450,)

Generating Sample 109
removed num of points 79
num of points remained 444
Set Hyper Parameters, solving for 10 agents
Number of trainable probabilities 444
last three [-0.26667133  0.33672306  0.04327727]
Shape (10, 4) (450,)

Generating Sample 110
removed num of points 79
num of points remained 444
Set Hyper Parameters, solving for 10 agents
Number of trainable probabilities 435
last three [0.17377696 0.44411115 0.02488301]
Shape (10, 4) (450,)

Generating Sample 111
removed num of points 79
num of points remained 444
Set Hyper Parameters, solving for 10 agents
Number of trainable probabilities 429
last three [-0.47119085  0.2788772   0.04965528]
Shape (10, 4) (450,)

Generating Sample 112
removed num of points 79
num of points remained 444
Set Hyper Parameters, solving for 10 agents
Number of trainable probabilities 428
last three [0.36081345 0.56015741 0.00659218]
Shape (10, 4) 

Number of trainable probabilities 447
last three [0.01639815 0.50806664 0.01827659]
Shape (10, 4) (450,)

Generating Sample 145
removed num of points 79
num of points remained 444
Set Hyper Parameters, solving for 10 agents
Number of trainable probabilities 444
last three [-0.4851069   0.26948914  0.0352876 ]
Shape (10, 4) (450,)

Generating Sample 146
removed num of points 79
num of points remained 444
Set Hyper Parameters, solving for 10 agents
Number of trainable probabilities 435
last three [-0.37504568  0.40217734  0.01967629]
Shape (10, 4) (450,)

Generating Sample 147
removed num of points 79
num of points remained 444
Set Hyper Parameters, solving for 10 agents
Number of trainable probabilities 429
last three [-0.13560364  0.2812342   0.0200621 ]
Shape (10, 4) (450,)

Generating Sample 148
removed num of points 79
num of points remained 444
Set Hyper Parameters, solving for 10 agents
Number of trainable probabilities 428
last three [-0.15808145  0.68215662  0.04524004]
Shape (1

Number of trainable probabilities 447
last three [-0.24000686  0.5804284   0.02040641]
Shape (10, 4) (450,)

Generating Sample 181
removed num of points 79
num of points remained 444
Set Hyper Parameters, solving for 10 agents
Number of trainable probabilities 444
last three [-0.34082411  0.59775842  0.01478391]
Shape (10, 4) (450,)

Generating Sample 182
removed num of points 79
num of points remained 444
Set Hyper Parameters, solving for 10 agents
Number of trainable probabilities 435
last three [0.14512708 0.55133309 0.00387366]
Shape (10, 4) (450,)

Generating Sample 183
removed num of points 79
num of points remained 444
Set Hyper Parameters, solving for 10 agents
Number of trainable probabilities 429
last three [0.32780949 0.46797423 0.03850634]
Shape (10, 4) (450,)

Generating Sample 184
removed num of points 79
num of points remained 444
Set Hyper Parameters, solving for 10 agents
Number of trainable probabilities 428
last three [-0.45650441  0.26252044  0.03550848]
Shape (10, 

Number of trainable probabilities 447
last three [-0.20646617  0.5923791   0.02925271]
Shape (10, 4) (450,)

Generating Sample 217
removed num of points 79
num of points remained 444
Set Hyper Parameters, solving for 10 agents
Number of trainable probabilities 444
last three [0.12962377 0.72289731 0.01985615]
Shape (10, 4) (450,)

Generating Sample 218
removed num of points 79
num of points remained 444
Set Hyper Parameters, solving for 10 agents
Number of trainable probabilities 435
last three [-0.03164295  0.25448046  0.00551347]
Shape (10, 4) (450,)

Generating Sample 219
removed num of points 79
num of points remained 444
Set Hyper Parameters, solving for 10 agents
Number of trainable probabilities 429
last three [0.45396753 0.42872763 0.04591102]
Shape (10, 4) (450,)

Generating Sample 220
removed num of points 79
num of points remained 444
Set Hyper Parameters, solving for 10 agents
Number of trainable probabilities 428
last three [-0.42058872  0.69031742  0.01409417]
Shape (10, 

In [12]:
# pd.DataFrame(exp_set.trainable_edges).to_csv("./data/gen-starts-ends/trainable_edges.csv", index=None)

In [ ]:
reload(genVoronoiDirected)

In [ ]:
X = pd.read_csv("./data/gen-starts-ends/X.csv")
X = X.to_numpy()
X = X.reshape((-1,5,4))

Y = pd.read_csv("./data/gen-starts-ends/Y.csv")
Y = Y.to_numpy()
Y = Y.reshape((-1,487))